# Imports

In [1]:
import pandas as pd
import numpy as np
import re
import json
from decimal import Decimal, getcontext

In [2]:
corrupcion_chile_df = pd.read_excel(io = './../modified_data/corrupcion_chile_bastian_olea/casos_corrupcion_chile.xlsx')
corrupcion_chile_df

,Caso,Monto,Año,Responsable,Sector,Partido,Año inicio,Año fin,Comuna,Posición,...,Delitos,Estado,Sentencia,Condena,Conclusión,Fuente1,Fuente2,Fuente3,Fuente4,Fuente5
0,Alcaldesa de Viña del Mar (Virginia Reginato),1.000000e+11,2021,Virginia Reginato,Derecha,UDI,2016.0,2021.0,Viña del Mar,Alcalde,...,"fraude al fisco, abandono de deberes",NaN,NaN,NaN,NaN,https://www.biobiochile.cl/noticias/nacional/r...,https://www.publimetro.cl/noticias/2022/09/23/...,https://www.fastcheck.cl/2023/05/16/misterio-s...,NaN,NaN
1,"Fraude en Carabineros, Pacogate",3.500000e+10,2016,"Eduardo Gordon, Gustavo González, Bruno Villal...",Ninguno,NaN,2006.0,2017.0,NaN,Carabineros,...,malversación de caudales públicos,NaN,NaN,NaN,NaN,https://es.wikipedia.org/wiki/Pacogate#Investi...,NaN,NaN,NaN,NaN
2,Alcaldesa de Maipú (Cathy Barriga),3.100000e+10,2024,Cathy Barriga,Derecha,UDI,NaN,NaN,Maipú,Alcalde,...,fraude al fisco y falsificación de instrumento...,NaN,NaN,NaN,prisión preventiva,https://radio.uchile.cl/2024/01/17/en-suspenso...,NaN,NaN,NaN,NaN
3,Caso Penta,3.010000e+10,2014,"Jovino Novoa, Carlos Alberto Délano",Derecha,UDI,2014.0,NaN,NaN,Privado,...,"fraude tributario, emisión de facturas ideológ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Caso Riggs (Pinochet),2.680000e+10,2014,NaN,Derecha,NaN,NaN,NaN,NaN,Presidente,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,"Director general de PDI, Héctor Espinosa",1.460000e+08,2021,Héctor Espinosa,Ninguno,NaN,2015.0,2017.0,NaN,Policía de Investigaciones,...,"malversación de caudales públicos, falsificaci...",NaN,NaN,NaN,NaN,https://www.latercera.com/nacional/noticia/for...,https://www.chvnoticias.cl/exclusivo-chv/todas...,NaN,NaN,NaN
83,Caso Luminarias: concejal de Iquique (Felipe A...,5.000000e+08,2024,Felipe Arenas,Derecha,UDI,2016.0,2018.0,Iquique,Concejal,...,"Fraude al fisco, cohecho reiterado",Condenado,Sí,Sí,"3 años y un día y 718 días de presidio, con el...",https://www.latercera.com/campuslt/noticia/lum...,https://iquiquetv.cl/2024/07/02/caso-luminaria...,NaN,NaN,NaN
84,"Caso Cuentas Corrientes, alcalde de San Ignaci...",NaN,2023,César Figueroa,Derecha,UDI,2021.0,2023.0,San Ignacio,Alcalde,...,"cohecho, falsificación de instrumento público,...",NaN,NaN,NaN,NaN,https://www.t13.cl/noticia/nacional/prision-pr...,https://www.elmostrador.cl/dia/2023/04/18/alca...,https://www.meganoticias.cl/nacional/415393-cu...,NaN,NaN
85,"Caso Cuentas Corrientes, alcalde de Ñiquén (Ma...",NaN,2023,Manuel Pino,Derecha,UDI,NaN,2023.0,Ñiquén,Alcalde,...,"malversación de caudales públicos, falsificaci...",NaN,NaN,NaN,NaN,https://www.cooperativa.cl/noticias/pais/regio...,NaN,NaN,NaN,NaN


In [3]:
ipc_variacion_anual_df = pd.read_csv('./../preprocessed_data/ipc_variacion_anual_junio.csv')
ipc_variacion_anual_df

,Periodo,IPC General,Reajuste IPC Acumulado
0,1929,1.8,1.018000e+00
1,1930,0.0,1.018000e+00
2,1931,-1.8,9.996760e-01
3,1932,1.8,1.017670e+00
4,1933,30.9,1.332130e+00
...,...,...,...
91,2020,2.6,2.782891e+08
92,2021,3.8,2.888641e+08
93,2022,12.5,3.249721e+08
94,2023,7.6,3.496700e+08


# Data Analysis

In [4]:
corrupcion_chile_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87 entries, 0 to 86
Data columns (total 24 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Caso         87 non-null     object 
 1   Monto        84 non-null     float64
 2   Año          87 non-null     int64  
 3   Responsable  70 non-null     object 
 4   Sector       85 non-null     object 
 5   Partido      67 non-null     object 
 6   Año inicio   41 non-null     float64
 7   Año fin      39 non-null     float64
 8   Comuna       60 non-null     object 
 9   Posición     74 non-null     object 
 10  Beneficiado  7 non-null      object 
 11  Perjudicado  28 non-null     object 
 12  Objetivo     6 non-null      object 
 13  Fundación    15 non-null     object 
 14  Delitos      67 non-null     object 
 15  Estado       16 non-null     object 
 16  Sentencia    4 non-null      object 
 17  Condena      3 non-null      object 
 18  Conclusión   15 non-null     object 
 19  Fuente1   

In [5]:
ipc_variacion_anual_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 3 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Periodo                 96 non-null     int64  
 1   IPC General             96 non-null     float64
 2   Reajuste IPC Acumulado  96 non-null     float64
dtypes: float64(2), int64(1)
memory usage: 2.4 KB


## Crimes Classification Standardization

In [6]:
def get_delitos_column_categories():
    delitos_categories = corrupcion_chile_df['Delitos'].dropna()
    delitos_categories = delitos_categories.to_list()
    print(f'N° of entries not null: {len(delitos_categories)}')
    
    categories_set = {}
    for line in delitos_categories:
        line_parsed = line.replace('\xa0', ' ')
        line_parsed = line_parsed.lower()
        line_categories = re.split(', |y ', line_parsed)
        for i in range(len(line_categories)):
            line_categories[i] = line_categories[i].rstrip().lstrip()
        try:
            categories_set.update(line_categories)
        except ValueError:
            categories_set = set(line_categories)

    print(f'N° of categories: {len(categories_set)}')
    return list(categories_set)

get_delitos_column_categories()

N° of entries not null: 67
N° of categories: 52


['compra de acciones con información privilegiada',
 'malversación de recursos públicos',
 'falsificación de instrumento público',
 'asociación ilícita',
 'boletas ideológicamente falsas',
 'pagos de horas extra',
 'delitos tributarios',
 'apropiación indebida',
 'aplicación pública diferente',
 'abandono de deberes',
 'cohecho reiterado',
 'pago de horas extra',
 'filtración de información secreta',
 'tráfico de información confidencial para tomar posiciones en activos financieros',
 'fraude tributario',
 'adjudicación de fondos a fundación sin experiencia ni giro adecuado',
 'facturas',
 'usurpación de identidad',
 'fraude al fisco reiterado',
 'falsificación de documento oficial',
 'estafa',
 'negociación incompatible',
 'administración desleal',
 'fraude al fisco',
 'déficit financiero',
 'emisión de boletas falsas',
 'ejecución incompleta de programa',
 'boletas sin respaldo',
 'asesorías jurídicas por trato directo',
 'malversación de caudales públicos',
 'malversación de fondos'

The categories identified above will be grouped according to the prosecution areas of [Corruption](http://www.fiscaliadechile.cl/Fiscalia/areas/cor-delitos.jsp), [Economic Crimes](http://www.fiscaliadechile.cl/Fiscalia/areas/eco-delitos.jsp) and [Organized crime and money laundering](http://www.fiscaliadechile.cl/Fiscalia/areas/org-delitos.jsp) of the Chilean Public Prosecutor's Office and the document [Delitos funcionarios contra la probidad administrativa: Chile y los casos de Argentina, España, Francia.](https://obtienearchivo.bcn.cl/obtienearchivo?id=repositorio/10221/33079/2/BCN_Delitos_funcionarios_contra_probidad._Definicion__catalogo_y_ultimas_modificaciones_Edit._2022.pdf), which focuses on technical parliamentary advisory. This classification is done manually based on these definition by a non-expert, so it may contain errors.

In [7]:
crimes_classification = {
    'Corrupción': {
        'Soborno': [
            'soborno',
            'financiamiento irregular de políticos',
        ],
        'Cohecho': [
            'cohecho',
            'cohecho reiterado'
        ],
        'Malversación': [
            'malversación',
            'malversación de recursos públicos',
            'malversación de fondos públicos',
            'malversación de caudales públicos',
            'malversación de fondos',
            'desfalco',
            'apropiación indebida',
            'enriquecimiento ilícito',
            'aplicación pública diferente',
            'adjudicación de fondos a fundación sin experiencia ni giro adecuado',
            'saldos no rendidos ni justificados',
            'asesorías jurídicas por trato directo'
        ],
        'Fraude al Fisco': [
            'fraude al fisco',
            'fraude fiscal',
            'fraude al fisco reiterado',
            'fraude de subvenciones estatales',
            'fraude de subvenciones',
            'negociaciones incompatibles',
            'negociación incompatible',
            'tráfico de influencia',
            'estafa',
            'pagos de horas extra',
            'pago de horas extra',
            'asignación de horas extras',
            'pagos de honorarios',
            'administración desleal',
        ],
        'Violación de secretos': [
            'violación de secreto',
            'filtración de información secreta',
            'tráfico de información confidencial para tomar posiciones en activos financieros',
            'compra de acciones con información privilegiada'
        ],
        'Prevaricación': [
            
        ],
    },
    'Delitos Económicos': {
        'Delitos Tributarios': [
            'boletas ideológicamente falsas',
            'emisión de boletas falsas',
            'emisión de boletas ideológicamente falsas',
            'emisión de facturas ideológicamente falsas',
            'uso malicioso de instrumento mercantil',
            'facturas',
            'boletas sin respaldo',
            'fraude tributario',
            'delitos tributarios'
        ],
        'Delitos Aduaneros': [
            
        ],
        'Delitos contra la Propiedad Intelectual': [
            
        ],
        'Fraudes': [
            
        ],
        'Delitos Informáticos': [
            
        ]
    },
    'Crimen Organizado y lavado de dinero': {
        'Lavado De Dinero': [
            'lavado de activos',
        ],
        'Asociación Ilícita': [
            'asociación ilícita',
        ],
        'Terrorismo': [
            
        ],
        'Armas': [
            
        ],
        'Trata y Tráfico': [
            
        ]
    },
    'Otros': [
        'uso de pasaportes falsos',
        'falsificación de instrumento público',
        'falsificación de documento oficial',
        'usurpación de identidad',      
        'abandono de deberes',
        'déficit financiero',
        'ejecución incompleta de programa',
    ]  
}

# Manual classification, the following lines help to identify the crimes ("delitos") not classified yet
crimes_set = set(get_delitos_column_categories())
print('\n')

already_classified_crimes_list = []
for prosecution_area_key in crimes_classification.keys():
    prosecution_area_sub_dict = crimes_classification[prosecution_area_key]
    try:
        for crimes_group_key in prosecution_area_sub_dict.keys():
            crimes_group_sub_list = prosecution_area_sub_dict[crimes_group_key]
            for crime in crimes_group_sub_list:
                already_classified_crimes_list.append(crime)
    except AttributeError:
        prosecution_area_sub_list = crimes_classification[prosecution_area_key]
        for crime in prosecution_area_sub_list:
            already_classified_crimes_list.append(crime)

already_classified_crimes_set = set(already_classified_crimes_list)
unclassified_crimes_set = crimes_set - already_classified_crimes_set
print(f'Already classified: {already_classified_crimes_set}\n')
print(f'Not classified yet: {unclassified_crimes_set}')

N° of entries not null: 67
N° of categories: 52


Already classified: {'malversación de recursos públicos', 'compra de acciones con información privilegiada', 'falsificación de instrumento público', 'boletas ideológicamente falsas', 'pagos de horas extra', 'delitos tributarios', 'asociación ilícita', 'apropiación indebida', 'aplicación pública diferente', 'cohecho reiterado', 'abandono de deberes', 'pago de horas extra', 'filtración de información secreta', 'tráfico de información confidencial para tomar posiciones en activos financieros', 'facturas', 'adjudicación de fondos a fundación sin experiencia ni giro adecuado', 'fraude al fisco reiterado', 'fraude tributario', 'usurpación de identidad', 'falsificación de documento oficial', 'estafa', 'negociación incompatible', 'administración desleal', 'fraude al fisco', 'déficit financiero', 'emisión de boletas falsas', 'asesorías jurídicas por trato directo', 'boletas sin respaldo', 'ejecución incompleta de programa', 'malversación de caud

In [8]:
# define inverted dict to optimize access to classes
inverted_crimes_classification_dict = {}

for prosecution_area_key in crimes_classification.keys():
    prosecution_area_sub_dict = crimes_classification[prosecution_area_key]
    try:
        for crimes_group_key in prosecution_area_sub_dict.keys():
            crimes_group_sub_list = prosecution_area_sub_dict[crimes_group_key]
            for crime in crimes_group_sub_list:
                inverted_crimes_classification_dict[crime] = (prosecution_area_key, crimes_group_key)               
                    
    except AttributeError:
        prosecution_area_sub_list = crimes_classification[prosecution_area_key]
        for crime in prosecution_area_sub_list:
            inverted_crimes_classification_dict[crime] = (prosecution_area_key, None) 
            
print(inverted_crimes_classification_dict)

{'soborno': ('Corrupción', 'Soborno'), 'financiamiento irregular de políticos': ('Corrupción', 'Soborno'), 'cohecho': ('Corrupción', 'Cohecho'), 'cohecho reiterado': ('Corrupción', 'Cohecho'), 'malversación': ('Corrupción', 'Malversación'), 'malversación de recursos públicos': ('Corrupción', 'Malversación'), 'malversación de fondos públicos': ('Corrupción', 'Malversación'), 'malversación de caudales públicos': ('Corrupción', 'Malversación'), 'malversación de fondos': ('Corrupción', 'Malversación'), 'desfalco': ('Corrupción', 'Malversación'), 'apropiación indebida': ('Corrupción', 'Malversación'), 'enriquecimiento ilícito': ('Corrupción', 'Malversación'), 'aplicación pública diferente': ('Corrupción', 'Malversación'), 'adjudicación de fondos a fundación sin experiencia ni giro adecuado': ('Corrupción', 'Malversación'), 'saldos no rendidos ni justificados': ('Corrupción', 'Malversación'), 'asesorías jurídicas por trato directo': ('Corrupción', 'Malversación'), 'fraude al fisco': ('Corrup

## Identify Classes in Categorical Columns

In [9]:
def find_unique_values_from_columns(columns_names):
    unique_values_dict = {}
    for column in columns_names:
        column_df = corrupcion_chile_df[column]
        column_df = column_df.dropna()
        unique_values = column_df.unique()
        if column == 'Partido':
            parsed_set = set()
            for string in unique_values:
                for value in string.split(', '):
                    parsed_set.add(value)
            unique_values_dict[column] = list(parsed_set)
            continue
        unique_values_dict[column] = list(unique_values)
    return unique_values_dict

def print_resume_from_dict(dict):
    for key, value in dict.items():
        print(f'[{key:12}]({len(value):2}): {value}\n')

In [10]:
categorical_columns = [
    'Sector',
    'Partido',
    'Posición',
    'Beneficiado',
    'Perjudicado',
    'Objetivo',
    'Estado'
]

categorical_unique_values = find_unique_values_from_columns(categorical_columns)
print_resume_from_dict(categorical_unique_values)

[Sector      ]( 4): ['Derecha', 'Ninguno', 'Centro', 'Izquierda']

[Partido     ](14): ['PRO', 'PC', 'UDI', 'PR', 'RN', 'Evópoli', 'Republicanos', 'PPD', 'Liberal', 'PS', 'DC', 'Comunes', 'RD', 'Independiente']

[Posición    ](12): ['Alcalde', 'Carabineros', 'Privado', 'Presidente', 'Senador', 'Fundación', 'Funcionario municipal', 'Subsecretario', 'Ministro', 'Diputado', 'Policía de Investigaciones', 'Concejal']

[Beneficiado ]( 6): ['Privados', 'Político', 'Alcalde', 'Campaña política', 'Partido', 'Personal']

[Perjudicado ]( 3): ['Municipio', 'Privado', 'Fisco']

[Objetivo    ]( 3): ['Financiamiento político', 'Enriquecimiento', 'Campaña política']

[Estado      ]( 9): ['En investigación', 'Suspensión condicional de la causa', 'Sentenciado', 'Sentencia', 'Formalizado', 'Prisión preventiva', 'Condenado', 'Responsable pero sin condena', 'Condenada']



In [11]:
def simplify_estado_column_categories(original_category):
    if original_category in ['Sentenciado', 'Sentencia']:
        return 'Sentencia'
    if original_category in ['Condenada', 'Condenado']:
        return 'Condena'
    return original_category
    
corrupcion_chile_df['Estado'] = corrupcion_chile_df['Estado'].apply(simplify_estado_column_categories)

categorical_unique_values = find_unique_values_from_columns(categorical_columns)
print_resume_from_dict(categorical_unique_values)

[Sector      ]( 4): ['Derecha', 'Ninguno', 'Centro', 'Izquierda']

[Partido     ](14): ['PRO', 'PC', 'UDI', 'PR', 'RN', 'Evópoli', 'Republicanos', 'PPD', 'Liberal', 'PS', 'DC', 'Comunes', 'RD', 'Independiente']

[Posición    ](12): ['Alcalde', 'Carabineros', 'Privado', 'Presidente', 'Senador', 'Fundación', 'Funcionario municipal', 'Subsecretario', 'Ministro', 'Diputado', 'Policía de Investigaciones', 'Concejal']

[Beneficiado ]( 6): ['Privados', 'Político', 'Alcalde', 'Campaña política', 'Partido', 'Personal']

[Perjudicado ]( 3): ['Municipio', 'Privado', 'Fisco']

[Objetivo    ]( 3): ['Financiamiento político', 'Enriquecimiento', 'Campaña política']

[Estado      ]( 7): ['En investigación', 'Suspensión condicional de la causa', 'Sentencia', 'Formalizado', 'Prisión preventiva', 'Condena', 'Responsable pero sin condena']



# Define production JSON

In [12]:
corrupcion_chile_json = {
    'metadata': {
        'country': 'Chile',
        'currency': 'Chilean Peso (CLP)',
        'inflation readjustment reference date': '2024-06',
        'crimes classification': {},
        'categorical classes': {}
    },
    'data': []
}

In [13]:
crimes_classes = {
    'Corrupción': [
        'Soborno',
        'Cohecho',
        'Malversación',
        'Fraude al Fisco',
        'Violación de secretos',
        'Prevaricación'
    ],
    'Delitos Económicos': [
        'Delitos Tributarios',
        'Delitos Aduaneros',
        'Delitos contra la Propiedad Intelectual',
        'Fraudes',
        'Delitos Informáticos'
    ],
    'Crimen Organizado y lavado de dinero': [
        'Lavado De Dinero',
        'Asociación Ilícita',
        'Terrorismo',
        'Armas',
        'Trata y Tráfico'
    ],
    'Otros': []
}

In [14]:
corrupcion_chile_json['metadata']['categorical classes'] = categorical_unique_values
corrupcion_chile_json['metadata']['categorical classes']['Delitos'] = crimes_classes

In [15]:
corrupcion_chile_json

{'metadata': {'country': 'Chile',
  'currency': 'Chilean Peso (CLP)',
  'inflation readjustment reference date': '2024-06',
  'crimes classification': {},
  'categorical classes': {'Sector': ['Derecha',
    'Ninguno',
    'Centro',
    'Izquierda'],
   'Partido': ['PRO',
    'PC',
    'UDI',
    'PR',
    'RN',
    'Evópoli',
    'Republicanos',
    'PPD',
    'Liberal',
    'PS',
    'DC',
    'Comunes',
    'RD',
    'Independiente'],
   'Posición': ['Alcalde',
    'Carabineros',
    'Privado',
    'Presidente',
    'Senador',
    'Fundación',
    'Funcionario municipal',
    'Subsecretario',
    'Ministro',
    'Diputado',
    'Policía de Investigaciones',
    'Concejal'],
   'Beneficiado': ['Privados',
    'Político',
    'Alcalde',
    'Campaña política',
    'Partido',
    'Personal'],
   'Perjudicado': ['Municipio', 'Privado', 'Fisco'],
   'Objetivo': ['Financiamiento político',
    'Enriquecimiento',
    'Campaña política'],
   'Estado': ['En investigación',
    'Suspensión con

## Identify Binary Columns

In [16]:
bolean_columns = [
    'Sentencia',
    'Condena'
]

## Identify Key Columns

In [17]:
key_columns = [
    'Caso'
]

## Identify Integer Columns

In [18]:
integer_columns = [
    'Monto',
    'Año',
    'Año inicio',
    'Año fin'
]

## Identify Additional Information Columns

In [19]:
additional_information_columns = [
    'Responsable',
    'Comuna',
    'Fundación',
    'Conclusión',
]

# consider all the columns Fuente(n)
fuente_columns = list(filter(lambda x : 'Fuente' in x, corrupcion_chile_df.columns))

## Format JSON Data

In [20]:
def calcule_inflation_readjustment(year):
    getcontext().prec = 16
    accumulative_readjustment_current_year = Decimal(ipc_variacion_anual_df[ipc_variacion_anual_df['Periodo'] == 2024]['Reajuste IPC Acumulado'].iloc[0])
    accumulative_readjustment_discount = Decimal(ipc_variacion_anual_df[ipc_variacion_anual_df['Periodo'] == year]['Reajuste IPC Acumulado'].iloc[0])
    accumulative_readjustment_value = accumulative_readjustment_current_year / accumulative_readjustment_discount
    return accumulative_readjustment_value

calcule_inflation_readjustment(2021)

Decimal('1.261341000000000')

In [21]:
corrupcion_chile_json['data'] = []

def format_crimes_for_json(row):
    pass

def format_row_for_json(row):
    row_formatted = {}

    string_columns = key_columns.copy()
    string_columns.extend(categorical_columns)
    string_columns.extend(additional_information_columns)

    for column in string_columns:
        if pd.isna(row[column]):
            row_formatted[column] = None
            continue

        if column in ['Responsable', 'Partido']:
            row_formatted[column] = row[column].split(', ') 
        else:
            row_formatted[column] = row[column] 

    for column in bolean_columns:
        if pd.isna(row[column]):
            row_formatted[column] = None
            continue

        if row[column] == 'Sí':
            row_formatted[column] = True
        else:
            row_formatted[column] = False

    for column in integer_columns:
        if pd.isna(row[column]):
            row_formatted[column] = None
            continue

        if  column == 'Monto':
            value = int(row[column])
            row_formatted['Monto'] = value  
            year = row['Año']
            inflation_readjustment = calcule_inflation_readjustment(year)
            readjusted_value = int(inflation_readjustment * value)
            row_formatted['Reajuste inflación'] = float(inflation_readjustment)
            row_formatted['Monto reajustado'] = readjusted_value
        else:
            row_formatted[column] = int(row[column])  

    row_formatted['Fuente'] = []
    for column in fuente_columns:
        if pd.isna(row[column]):
            break
        else:
            row_formatted['Fuente'].append(row[column])  

    
    row_formatted['Delitos'] = {}
    try:
        crimes_list = re.split(', |y ', row['Delitos'])
        crimes_list = [crime.rstrip().lstrip().replace('\xa0', ' ').lower() for crime in crimes_list]
        row_formatted['Delitos detalle'] = crimes_list
        for crime in crimes_list:
            prosecution_area, crime_type = inverted_crimes_classification_dict[crime]    
            try:
                if not crime_type in row_formatted['Delitos'][prosecution_area]:
                    row_formatted['Delitos'][prosecution_area].append(crime_type)
            except KeyError:
                row_formatted['Delitos'][prosecution_area] = [crime_type]

    except TypeError:
        row_formatted['Delitos detalle'] = []

    corrupcion_chile_json['data'].append(row_formatted)

corrupcion_chile_df.apply(format_row_for_json, axis = 1)
corrupcion_chile_json['data'][0:2]

[{'Caso': 'Alcaldesa de Viña del Mar (Virginia Reginato)',
  'Sector': 'Derecha',
  'Partido': ['UDI'],
  'Posición': 'Alcalde',
  'Beneficiado': None,
  'Perjudicado': 'Municipio',
  'Objetivo': None,
  'Estado': None,
  'Responsable': ['Virginia Reginato'],
  'Comuna': 'Viña del Mar',
  'Fundación': None,
  'Conclusión': None,
  'Sentencia': None,
  'Condena': None,
  'Monto': 100000000000,
  'Reajuste inflación': 1.261341,
  'Monto reajustado': 126134100000,
  'Año': 2021,
  'Año inicio': 2016,
  'Año fin': 2021,
  'Fuente': ['https://www.biobiochile.cl/noticias/nacional/region-de-valparaiso/2022/09/23/auditoria-revela-caos-municipal-en-administracion-reginato-100-mil-millones-en-perdidas.shtml',
   'https://www.publimetro.cl/noticias/2022/09/23/100-mil-millones-las-perdidas-de-virginia-reginato-en-municipalidad-de-vina-del-mar/',
   'https://www.fastcheck.cl/2023/05/16/misterio-sin-resolver-ocho-escandalos-de-corrupcion-municipal-pendientes-de-justicia/'],
  'Delitos': {'Corrupción

## Save JSON file

In [22]:
#json_file = json.dump(corrupcion_chile_json, indent = 4, ensure_ascii = False)
#print(json_file)
with open('./../preprocessed_data/casos_corrupcion_chile.json', 'w') as file:
    json.dump(corrupcion_chile_json, file, indent = 4, ensure_ascii = False)